## **Integer Linear Programming**

![](2023-11-02-19-31-19.png)

![](2023-11-02-19-33-54.png)

![](2023-11-02-19-37-49.png)

![](2023-11-02-19-39-40.png)

![](2023-11-02-19-40-56.png)

![](2023-11-02-19-42-02.png)

![](2023-11-02-19-49-47.png)

**Once it is shown that it is NP-Hard, we would like to show it is NP-Complete:**

![](2023-11-02-19-51-11.png)

![](2023-11-02-19-57-37.png)

![](2023-11-02-19-58-16.png)

![](2023-11-02-19-59-12.png)

![](2023-11-02-20-00-39.png)

![](2023-11-04-17-33-07.png)

![](2023-11-04-17-35-11.png)

![](2023-11-04-17-38-24.png)

![](2023-11-04-17-40-59.png)

![](2023-11-04-19-35-01.png)

![](2023-11-05-07-48-00.png)

![](2023-11-05-07-56-37.png)

![](2023-11-05-07-56-52.png)

![](2023-11-05-07-57-32.png)

![](2023-11-05-07-57-46.png)

**Then bring the solutions together, and**

![](2023-11-05-07-59-09.png)

**You would choose that branhc solution as the best answer. If one leaf is infeasible:**

![](2023-11-05-08-06-09.png)

![](2023-11-05-08-07-27.png)

![](2023-11-05-08-08-07.png)

**One trick to help the program run faster is to keep track of the best solution so far, and when we get to a branch with objective function less than that we would not continue that branhc:**

![](2023-11-05-08-11-40.png)

## **01. Knapsack Problem**

In [1]:
from pulp import * # get all the definitions from pulp library 

def solve_candy_knapsack(n, candy_number_limits, candy_prices, candy_calories, N, Cmin, Cmax):
    assert len(candy_number_limits) == n, 'size mismatch'
    assert len(candy_prices) == n, 'size mismatch for prices'
    assert len(candy_calories) == n, 'size mismatch for list of calories'
    assert N >= 1, 'total number of candies per box must be 1 or more'
    assert Cmin <= Cmax, 'minimum calories is greater than the maximum calories'
    prob = LpProblem('Candy Knapsack', LpMinimize)
    # add decision variables
    # make a list of n decision variables, one for each candy. When declaring the variable, automatically declare
    # its lower bound to be 0 and upper bound to be ki from the candy_number_limits array
    # also declare the category (cat) of the variable to be integers.
    dVars = [LpVariable(f'x{i}',lowBound=0, upBound=ki, cat='Integer') for (i, ki) in enumerate(candy_number_limits)]
    # Now set the objective
    prob += lpSum([pj*xj for (pj,xj) in zip(candy_prices, dVars)])
    # Now add the constraints
    prob += lpSum(dVars) <= N # constraints on number of candies per box
    calories_of_candies = lpSum([cj*xj for (cj,xj) in zip(candy_calories, dVars)])
    prob += calories_of_candies <= Cmax
    prob += calories_of_candies >= Cmin
    status = prob.solve()
    if status == constants.LpStatusInfeasible:
        print('Problem is infeasible')
        return
    elif status == constants.LpStatusUnbounded:
        print('Problem is unbounded. Cannot proceed')
        return
    else:
        assert status == constants.LpStatusOptimal, 'Something went wrong while solving since status is either undefined or unsolved'
        # extract values
        print('Success: optimal answer found')
        solution_values = [x.varValue for x in dVars]
        for (j, svj) in enumerate(solution_values):
            print(f'\t Candy Type # {j}: {svj} candies')
        print(f'Total Cost: {sum([(pj*svj) for (pj, svj) in zip(candy_prices, solution_values)])}')
        print(f'Calories: {sum([cj*xj for (cj,xj) in zip(candy_calories, solution_values)])}')

n = 5
candy_number_limits = [10, 12, 10, 11, 10]
candy_prices = [0.2, 0.5, 0.1, 0.4, 0.8]
candy_calories = [25, 12, 22, 14, 33]
solve_candy_knapsack(5, candy_number_limits, candy_prices, candy_calories, 12, 250, 500)        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/3e2b8775f49e4ea9999ea433016be8ca-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/3e2b8775f49e4ea9999ea433016be8ca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 39 RHS
At line 43 BOUNDS
At line 49 ENDATA
Problem MODEL has 3 rows, 5 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.24 - 0.00 seconds
Cgl0004I processed model has 2 rows, 5 columns (5 integer (0 of which binary)) and 10 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0012I Integer solution of 1.7 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds

## **02. Set Cover Problem: Invitation Planning**

![](2023-11-17-18-37-20.png)

In [2]:
def plan_invite_list(n, m, T_lists, G_lists, pp_scores):
    assert m >= 0, 'Cannot invite a negative number of people'
    assert all( 0 <= j < n for ti in T_lists for j in ti ) # check that all employee IDs are valid
    assert all( 0<= j < n for gi in G_lists for j in gi)
    assert len(pp_scores) == n, 'Length of party pooper scores list does not match number of employees'
    prob = LpProblem('PartyPlanner', LpMinimize)
    # create decision variables
    dvars = [LpVariable(f'w_{i}', cat='Binary') for i in range(n)] # declare variables as binary
    # if we declared variables as binary they are automatically treated as either 0 or 1 in the optimization 
    # create objective
    prob += lpSum([si * wi for (si, wi) in zip(pp_scores, dvars)])
    # limit on number of invitees
    prob += sum(dvars) >= m
    # at least two people from each team
    for ti in T_lists:
        prob += lpSum([dvars[j] for j in ti]) >= len(ti)/4
    # no more than one person per grievance set
    for gj in G_lists:
        prob += lpSum(dvars[j] for j in gj) <= 1
    # solve and get the result
    status = prob.solve()
    if status == constants.LpStatusInfeasible:
        print('infeasible LP')
        return 
    elif status != constants.LpStatusOptimal:
        print('Unbounded or undefined LP Status -- there is some mistake in the problem formulation since it cannot happen in theory')
        return 
    else: 
        assert status == constants.LpStatusOptimal
        # extract values
        sol = [x.varValue for x in dvars]
        for (j,wj) in enumerate(sol):
            if wj >= 1:
                print(f'Invite person {j}')
        print(f'Total # of invitees: {sum(sol)}')
        for j,tj in enumerate(T_lists):
            print(f'# of invitees from Team # {j}: {sum([sol[k] for k in tj])}')
        for k, gk in enumerate(G_lists):
            print(f'# of invitees from Grievance set # {k}: {sum([sol[j] for j in gk])}')
        return
    
n = 20
m = 12
T_lists = [[1, 5, 12, 18, 19], [2, 3, 4, 6, 7], [1, 2, 4, 7, 8, 9, 10, 11, 12, 14, 16], [1, 3, 4, 5, 6, 13, 15, 17, 18, 19], [1, 5, 7, 8,9, 19]]
G_lists = [[1, 5], [5, 19], [4, 7], [4, 12], [4, 19], [4, 18], [3, 4, 15, 19], [4, 7, 18, 2]]
pp_scores = [1, 2, 2, 1, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3.5, 1, 0.6, 0, 1, 8, 8]
plan_invite_list(n, m, T_lists, G_lists, pp_scores)    

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/79df2fa86a74457d98f2d9d9630f4958-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/79df2fa86a74457d98f2d9d9630f4958-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19 COLUMNS
At line 156 RHS
At line 171 BOUNDS
At line 192 ENDATA
Problem MODEL has 14 rows, 20 columns and 77 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18.1 - 0.00 seconds
Cgl0003I 1 fixed, 0 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I proce

## **03. LP Relaxation**

min 2x1​−3x2​+x3

s.t.

​x1​−x2​≥0.5

x1​−x2​≤0.75

x2​−x3​≤1.25

x2​−x3​≥0.95

x1​,x2​,x3​∈[−1,1]

x1​,x2​,x3​∈Z​

In [5]:
from pulp import *

# Create a minimization problem
problem = LpProblem("ILP_Minimization", LpMinimize)

# Define the decision variables with lower and upper bounds as integers
x1 = LpVariable("x1", lowBound=-1, upBound=1, cat=LpInteger)
x2 = LpVariable("x2", lowBound=-1, upBound=1, cat=LpInteger)
x3 = LpVariable("x3", lowBound=-1, upBound=1, cat=LpInteger)

# Define the objective function
objective_function = 2 * x1 - 3 * x2 + x3
problem += objective_function

# Define the constraints
problem += x1 - x2 >= 0.5
problem += x1 - x2 <= 0.75
problem += x2 - x3 <= 1.25
problem += x2 - x3 >= 0.95

# Solve the problem
problem.solve()
print('# ---------------------ILP Solution--------------------------')

# Print the results
if problem.status == LpStatusOptimal:
    print("Optimal Solution Found:")
    print(f"x1 = {x1.varValue}")
    print(f"x2 = {x2.varValue}")
    print(f"x3 = {x3.varValue}")
    print(f"Objective Value = {value(problem.objective)}")
elif problem.status == constants.LpStatusInfeasible:
    print('Problem is infeasible')
elif problem.status == constants.LpStatusUnbounded:
    print('Problem is unbounded. Cannot proceed')


print('# ---------------------LP Relaxation-------------------------')

# Create a linear programming (LP) relaxation problem
problem = LpProblem("LP_Relaxation", LpMinimize)

# Define the decision variables with lower and upper bounds
x1 = LpVariable("x1", lowBound=-1, upBound=1)
x2 = LpVariable("x2", lowBound=-1, upBound=1)
x3 = LpVariable("x3", lowBound=-1, upBound=1)

# Define the objective function
objective_function = 2 * x1 - 3 * x2 + x3
problem += objective_function, "Objective Function"

# Define the constraints
problem += x1 - x2 >= 0.5, "Constraint 1"
problem += x1 - x2 <= 0.75, "Constraint 2"
problem += x2 - x3 <= 1.25, "Constraint 3"
problem += x2 - x3 >= 0.95, "Constraint 4"

# Solve the LP relaxation problem
problem.solve()

# Print the results
if problem.status == LpStatusOptimal:
    print("Optimal Solution Found:")
    print(f"x1 = {x1.varValue}")
    print(f"x2 = {x2.varValue}")
    print(f"x3 = {x3.varValue}")
    print(f"Objective Value = {value(problem.objective)}")
elif problem.status == constants.LpStatusInfeasible:
    print('Problem is infeasible')
elif problem.status == constants.LpStatusUnbounded:
    print('Problem is unbounded. Cannot proceed')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/f8a6c16ce438402db471fd2374c46275-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/70/jz6cvvk97t30vggrppkktlzh0000gn/T/f8a6c16ce438402db471fd2374c46275-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 27 RHS
At line 32 BOUNDS
At line 39 ENDATA
Problem MODEL has 4 rows, 3 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -0.25 - 0.00 seconds
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.02

# ----------

## **04. Branch and Bound with Pruning**

In [8]:
class Stats:
    def __init__(self):
        self.max_depth = 0
        self.num_lps = 0
        self.num_infeas = 0
        
    def print(self):
        print('Statistics')
        print('---------------------------------')
        print(f'Number of LPs Solved: {self.num_lps}')
        print(f'Number of infeasible LPs: {self.num_infeas}')
        print(f'Max. Recursion Depth: {self.max_depth}')
        print('---------------------------------')

In [9]:
from pulp import *
from math import floor, ceil

# Given a list of numbers, find all indices in the list that correspond to fractional numbers.
# we use the function is_fractional to define what it means to be "fractional" -- we have to 
# account for floating point errors in this criterion
def find_all_fractional_indices(sols):
    n = len(sols)
    def is_fractional(f):
        epsilon = 0.001 # this is the precision to which we check whether something is an integer
        return  epsilon <= (f - floor(f)) <= 1-epsilon
    return [j for j in range(n) if is_fractional(sols[j]) ]


class Solution:
    # a class that will hold the best solution so far
    def __init__(self):
        self.obj_val = -float('inf') # initialize it to -infinity
        self.sol = None
    
    # replace current solution with a new one if it is better
    def replace(self, obj_val, new_sol):
        if obj_val > self.obj_val:
            self.obj_val = obj_val
            self.sol = new_sol
        return
    
# Branch and Bound with Pruning.
# The logic is almost identical to Branch and Bound but we pass around an extra argument
# that tracks the best solution so far
# We pass this argument as a class so that the reference to the class is passed. This allows
# changes made to the best solution so far in one branch to reflect elsewhere in the code.
# If you are unaware of this aspect of python, read about it here: https://www.geeksforgeeks.org/pass-by-reference-vs-value-in-python/#

def branch_and_bound_pruning_solve(c, A, b, lb, ub, best_sol_so_far, stats, depth=1, debug=True):
    n = len(c)
    m = len(A)
    assert all( len(ai) == n for ai in A), 'All rows in A must have same length'
    assert len(b) == m
    assert len(lb) == n
    assert len(ub) == n
    ## This function is just for pretty printing the tabs
    def indent():
        for _ in range(depth-1):
            print('|  ', end='')
        print(u'\u21B3', end='')
    # Check if all lower bounds are <= upper bounds, if not exit
    if any(lb[i] > ub[i] for i in range(n)): # if any lower bound is greater than upper bound, problem is infeasible, BAIL
        if debug:
            indent()
            print('Infeasibility detected due to lb[i]> ub[i].')
        return (-float('inf'), None)
    # record statistics
    if depth > stats.max_depth:
        stats.max_depth = depth
    stats.num_lps = stats.num_lps + 1
    # setup and solve the LP
    prob = LpProblem('b_and_b', LpMaximize) # create the problem
    # declare all the variables, set the upper and lower bound here
    # Note: variables are set to "Continuous" since we are solving the LP relaxation
    dvars = [LpVariable(f'x{i}', lowBound=lb[i], upBound=ub[i],cat='Continuous') for i in range(n)]
    # add the objective function -- we already set it to maximize
    prob += lpSum([ci*xi for (ci,xi) in zip(c, dvars)])
    # add the constraints for each row
    for (ai, bi) in zip(A,b):
        prob += lpSum([ aij*xj for (aij,xj) in zip(ai, dvars)]) <= bi
    #Let's solve the problem but supress the output
    status = prob.solve(apis.PULP_CBC_CMD(mip=False, msg=False))
    # Check the problem status
    if status == constants.LpStatusInfeasible: # infeasible?
        if debug:
            indent()
            print('Infeasible Problem: returning -\infty')
        stats.num_infeas = stats.num_infeas + 1
        return (-float('inf'), None) # return objective value of -infinity
    elif status == constants.LpStatusUnbounded: # unbounded
        if debug: 
            indent()
            print('Unbounded Problem: returning + \infty')
        return (float('inf'), None) # return objective value of +infinity
    else: # optimal?
        assert status == LpStatusOptimal, 'Problem results in undefined status -- cannot continue'
        sols = [x.varValue for x in dvars] # extract the solutions for each decision variable
        obj_value = sum([ci*xi for (ci, xi) in zip(c, sols)]) # compute objective value
        if debug:
            indent()
            print(f'Obtained solution: {sols} with objective {obj_value}')
        # is this objective less than the best so far?
        if obj_value < best_sol_so_far.obj_val: # if it is , then we do not need to go any further
            if debug:
                indent()
                print(f'Pruned solution: we already have an integer solution with objective {best_sol_so_far.obj_val}')
            return (-float('inf'), None) # it does not matter that we return a fake solution here since this will never beat the best solution so far.
 
        # check if the solution is integral
        jList = find_all_fractional_indices(sols)
        if len(jList) == 0: # no fractional values 
            if debug:
                indent()
                print('All variables are integral in optimal solution.')
                indent()
                print(f'Returning solution: {sols} with objective {obj_value}')
            if debug and obj_value > best_sol_so_far.obj_val:
                indent()
                print(f'This solution is the BEST SO FAR!!')
            best_sol_so_far.replace(obj_value, sols) # replace the best so far if appropriate.
            return (obj_value, sols) # done
        else:
            # solution is not integral
            split_var = jList[0]
            assert 0 <= split_var < n
            if debug:
                indent()
                print(f'Splitting variable : {split_var} which has a current solution {sols[split_var]}')
            # adjust upper bound 
            tmp = ub[split_var]
            ub[split_var] = floor(sols[split_var])
            if debug: 
                indent()
                print(f'Branch #1: Adding x{split_var} <= {ub[split_var]}')
            (obj1,sols1) = branch_and_bound_pruning_solve(c, A, b, lb, ub, best_sol_so_far,stats, depth+1, debug)
            if obj1 == float('inf'):
                return (obj1, sols1)
            ub[split_var] = tmp
            # adjust the lower bound
            tmp = lb[split_var]
            lb[split_var] = ceil(sols[split_var])
            if debug: 
                indent()
                print(f'Branch #2: Adding x{split_var} >= {lb[split_var]}')
            (obj2, sols2) = branch_and_bound_pruning_solve(c, A, b, lb, ub, best_sol_so_far, stats, depth+1, debug)
            lb[split_var] = tmp
            if obj2 == float('inf'): # unbounded 
                return (obj2, sols2)
            # compare solutions and return
            if obj1 < obj2: # which solution is larger?
                if debug:
                    indent()
                    print(f'Returning solution: {sols2} with objective {obj2}')
                return (obj2, sols2)
            else:
                if debug:
                    indent()
                    print(f'Returning solution: {sols1} with objective {obj1}')
                return (obj1, sols1)

$\begin{array}{rl}
\max & x_1 -2 x_2 +3 x_3 \\ 
\mathsf{s.t.} & x_1 - 3 x_2  \leq 7.1 \\ 
& 3 x_2 - x_3 \leq 4 \\ 
& 2 x_1 - 3 x_3 \leq 5 \\ 
& x_1 + x_2 + x_3 \leq 12.3 \\ 
& -15 \leq x_i \leq 15,\ i = 1,2, 3\\
\end{array}$

In [12]:
c = [1, -2, 3]
A = [[1, -3, 0], [0, 3, -1], [2, 0, -3], [1, 1, 1]]
b = [7.1, 4, 5, 12.3]
lb = [-15, -15, -15]
ub = [15, 15, 15]
stats = Stats()
branch_and_bound_pruning_solve(c, A, b, lb, ub, Solution(), stats)
stats.print()

↳Obtained solution: [-0.25, -2.45, 15.0] with objective 49.65
↳Splitting variable : 0 which has a current solution -0.25
↳Branch #1: Adding x0 <= -1
|  ↳Obtained solution: [-1.0, -2.7, 15.0] with objective 49.4
|  ↳Splitting variable : 1 which has a current solution -2.7
|  ↳Branch #1: Adding x1 <= -3
|  |  ↳Obtained solution: [-1.9, -3.0, 15.0] with objective 49.1
|  |  ↳Splitting variable : 0 which has a current solution -1.9
|  |  ↳Branch #1: Adding x0 <= -2
|  |  |  ↳Obtained solution: [-2.0, -3.0333333, 15.0] with objective 49.0666666
|  |  |  ↳Splitting variable : 1 which has a current solution -3.0333333
|  |  |  ↳Branch #1: Adding x1 <= -4
|  |  |  |  ↳Obtained solution: [-4.9, -4.0, 15.0] with objective 48.1
|  |  |  |  ↳Splitting variable : 0 which has a current solution -4.9
|  |  |  |  ↳Branch #1: Adding x0 <= -5
|  |  |  |  |  ↳Obtained solution: [-5.0, -4.0333333, 15.0] with objective 48.0666666
|  |  |  |  |  ↳Splitting variable : 1 which has a current solution -4.033333